# Подготовка

In [ ]:
import csv
import glob
import yaml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display, Markdown as md
from collections import OrderedDict
from datetime import datetime
from scrapinghub import ScrapinghubClient

In [ ]:
# загружаем конфиг
with open('config.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

# Загрузка данных

Данные выгружаются напрямую из Scrapinghub по API. Нужный датасет (или даже нужные датасеты, их можно быть много) ищутся по их тегу. Чтобы все работало, в конфиге тулбокса (файл `config.yaml`) нужно в параметре `category_dashboard:job_tag_filter` указать название тэга выгрузки Scrapinghub, по которой нужно выгружать данные. Скрипт выгрузит результаты работы всех завершенных задач с указанным тегом. Тег в Scrapinghub нужно указать при создании выгрузки данных (job).

Дальше этот датасет будет сохранен в папку data директории, в которой находится блокнт. Если в папке уже лежит файл с названием вида `category_dashboard_*.csv`, то новые данные загружены не будут.

Нужный файл можно положить и вручную, скачав его из Scrapinghub. Можно даже положить несколько файлов – скрипт возьмет последний по алфавиту.

После изменения переменной с тегом в Юпитере нужно выбрать пункт меню "Run -> Run All Cells". Дальше блокнот сделает все сам.

In [ ]:
# здесь нужно указать тэг нужной выгрузки, по-умолчанию он берется из конфига
job_tag_filter = config['category_dashboard']['job_tag_filter']

# остальное лучше не трогать
api_key = config['scrapinghub']['api_key']
project_id = config['scrapinghub']['project_id']
csv_name =  './data/category_dashboard_' + str(datetime.now()) + '.csv'

def load_data(job_tag_filter, api_key, project_id, csv_name):
    client = ScrapinghubClient(api_key)
    project = client.get_project(project_id)
    fieldnames = []
    
    jobs_summary = project.jobs.iter(has_tag=[job_tag_filter], state='finished')
    
    # определяем поля выгрузки
    for job in jobs_summary:
        for item in client.get_job(job['key']).items.iter():
            fieldnames = item.keys()
            break
        break
    
    # готовим CSV
    with open(csv_name, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        jobs_summary = project.jobs.iter(has_tag=[job_tag_filter], state='finished')

        for job in jobs_summary:
            for item in client.get_job(job['key']).items.iter():
                writer.writerow(item)

load_data(job_tag_filter, api_key, project_id, csv_name)

In [ ]:
pd.set_option("display.precision", 2)

latest_csv = sorted(glob.glob('./data/category_dashboard_*.csv'), reverse=True)[0]
data = pd.read_csv(latest_csv)

# сложная конструкция для исключения NaN значений цен (иногда проскакивают)
data = data[~np.isnan(data['wb_price'])]
data = data[~np.isnan(data['wb_purchases_count'])]

data['wb_turnover'] = data['wb_price'] * data['wb_purchases_count']

In [ ]:
data

data.loc[0].at['product_name']

# Статистика цен

Основные описательные статистики:

In [ ]:
md("Показатель | Значение\n--- | ---\nКоличество товаров | {}\nКоличество продаж | {}\nСредние продажи  | {}\nМедиана продаж | {}\nСамый дорогой (руб) | {}\nСамый дешевый (руб) | {}\nСредняя цена | {}".format(len(data.index), data['wb_purchases_count'].sum(), round(data['wb_purchases_count'].mean(), 2), round(data['wb_purchases_count'].median(), 2), data['wb_price'].max(), data['wb_price'].min(), round(data['wb_price'].mean(), 2)))

Распределение цен на товары в разделе:

In [ ]:
sns.distplot(data['wb_price'], rug=True)

# Статистика продаж

Распределение количества продаж:

In [ ]:
sns.distplot(data['wb_purchases_count'], rug=True)

# Статистика рейтингов

Распределение рейтингов:

In [ ]:
sns.distplot(data['wb_rating'])

Распределение количества отзывов:

In [ ]:
sns.distplot(data['wb_reviews_count'], rug=True)

# Парные зависимости

In [ ]:
g = sns.PairGrid(data, vars=["wb_rating", "wb_reviews_count", "wb_price", "wb_purchases_count"])
g.map(plt.scatter);